<b><u><font size="6">Data Structure</font></u></b>

In [1]:
import numpy as np
# rainy = 0
# dry = 1
# umbrella = 0
# no umbrella = 1

PI = np.array([.5, .5])
A = np.array([[.7, .3],[.3, .7]])
B = np.array([[.9, .1],[.2, .8]])
Observables = np.array([0, 0])

In [2]:
# depressed = 0
# healthy = 1

# movement = 0
# passive social networking = 1
# active social networking = 2
# texting = 3
# access psych health apps = 4

PI = np.array([.5, .5])
A = np.array([[.999, .001],[.2, .8]])
B = np.array([[0.05, 0.35, 0.2, 0.2, 0.2],[0.5, 0.1, 0.3, 0.1, 0]])
Observables = np.array([0, 3, 0, 3])



<b><u><font size="6">Filtering</font></u></b>

In [3]:
def forward(PI, A, B, Observables):
    alpha = np.zeros((PI.shape[0], Observables.shape[0]))
    
    for j in range(0, alpha.shape[1]):
        for i in range(0, alpha.shape[0]):
            if j == 0:
                alpha[i,j] = PI[i] * B[i,j]
            else:
                sum = 0
                for k in range(0, alpha.shape[0]):
                    sum += (alpha[k,j-1] * A[k,i])
                alpha[i,j] = sum * B[i,Observables[j]]                
    return alpha
    
def forward_normalized(PI, A, B, Observables):
    alpha = forward(PI, A, B, Observables)
    cArr = np.zeros(alpha.shape[1])
    for i in range(0, alpha.shape[1]):
        c = 0
        for j in range(0, alpha.shape[0]):
            c += alpha[j,i]
        cArr[i] = 1/c
        
    for i in range(0, alpha.shape[1]):
        for j in range(0, alpha.shape[0]):
            alpha[j,i] *= cArr[i]
    
    return alpha, cArr

In [4]:
a = forward(PI, A, B, Observables)
print("a1: ",a)

a, c = forward_normalized(PI, A, B, Observables)
print("\na2: ",a)
print("c2: ",c)

a1:  [[0.025      0.014995   0.00094903 0.00050996]
 [0.25       0.0200025  0.0080085  0.00064077]]

a2:  [[0.09090909 0.42845918 0.10594729 0.44315801]
 [0.90909091 0.57154082 0.89405271 0.55684199]]
c2:  [  3.63636364  28.57346953 111.6380084  869.01369787]


<b><u><font size="6">Evaluation</font></u></b>

In [5]:
def evaluation_unnormalized(alpha):
    cArr = np.zeros(alpha.shape[1])
    for i in range(0, alpha.shape[1]):
        c = 0
        for j in range(0, alpha.shape[0]):
            c += alpha[j,i]
        cArr[i] = 1/c
        
    eval = 1
    for i in range(0, cArr.shape[0]):
        eval *= cArr[i]
    eval = 1/eval
    
    return eval

In [6]:
a = forward(PI, A, B, Observables)
evaluation_unnormalized(a)

9.920441804712383e-08

<b><u><font size="6">Backward Probability</font></u></b>

In [7]:
def backward(PI, A, B, Observables):
    beta = np.zeros((len(Observables), PI.shape[0]))

    beta[len(Observables) - 1 , :] = 1.0

    for i in range(len(Observables) - 2, -1, -1):
        for j in range(PI.shape[0]):
            for k in range(PI.shape[0]):
                beta[i,j] += beta[i+1, k] * A[j,k] * B[k, Observables[i+1]]
    
    beta = np.rot90(beta)
    betaRot = np.ones((beta.shape[0], beta.shape[1]))
    height = beta.shape[0]
    print(height)
    for i in range(height):
        print("i:",i)
        print("I:",height-i-1)
        betaRot[i] = beta[height-i-1]
        
    return betaRot

def backward_normalized(PI, A, B, c, Observables):
    arr = backward(PI, A, B, Observables)
    
    beta = np.asarray(arr) 
            
    for i in range(0, beta.shape[1]):
        for j in range(0, beta.shape[0]):
            beta[j,i] *= c[i]
    
    return beta

In [8]:
b = backward(PI, A, B, Observables)
print("b1: ",b)

a,c = forward_normalized(PI, A, B, Observables)
b2 = backward_normalized(PI, A, B, c, Observables)
print("b2: ", b2)

2
i: 0
I: 1
i: 1
I: 0
b1:  [[0.00201199 0.010045   0.1999     1.        ]
 [0.00440172 0.049999   0.12       1.        ]]
2
i: 0
I: 1
i: 1
I: 0
b2:  [[7.31633418e-03 2.87020644e-01 2.23164379e+01 8.69013698e+02]
 [1.60062553e-02 1.42864490e+00 1.33965610e+01 8.69013698e+02]]


<b><u><font size="6">Smoothing by Forward-Backward Algorithm</font></u></b>

In [17]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    
    fwd_back = np.zeros((noOfStates, noOfTimeSteps))
    
    for i in range(0, noOfStates):
        for j in range(0, noOfTimeSteps):
            fwd_back[i][j] = alpha[i][j] * beta[i][j]
    
    return fwd_back

In [18]:
a,c = forward_normalized(PI, A, B, Observables)
b = backward_normalized(PI, A, B, c, Observables)
d = smoothed_probability(a,b,len(PI),len(Observables))

print("d:  ", d)

2
i: 0
I: 1
i: 1
I: 0
d:   [[6.65121289e-04 1.22976629e-01 2.36436609e+00 3.85110378e+02]
 [1.45511412e-02 8.16528886e-01 1.19772317e+01 4.83903320e+02]]
